## 데이터 확인하기

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings(action='ignore')


In [37]:
# place_number = {
#     '경주골굴사':['02089','02090'], '경주김유신묘':['01025','01036'], 
#     '경주동궁과월지':['01225', '01237'], '경주동부사적지대':['01279','01280'], 
#     '경주무열왕릉':['01194','01195'], '경주문무대왕릉':['02056','02057'],
#     '경주불국사':['01160','01161'], '경주석굴암':['01199','01199'], 
#     '경주신문왕릉':['01081','01082'], '경주월성(반월성)':['01237','01238'], 
#     '경주첨성대':['01279','01280'], '경주금곡사':['05379','05380'],
#     '경주기림사':['02021','02021'], '경주백률사':['01228','01229'],
#     '경주분황사':['01157','01158'],'경주연지암':['05423','05426'], 
#     '경주주사암':['03121','03122']
# } 
place_number = {
    '경주골굴사':['2089','2090'], '경주김유신묘':['1025','1036'], 
    '경주동궁과월지':['1225', '1237'], '경주동부사적지대':['1279','1280'], 
    '경주무열왕릉':['1194','1195'], '경주문무대왕릉':['2056','2057'],
    '경주불국사':['1160','1161'], '경주석굴암':['1199','1199'], 
    '경주신문왕릉':['1081','1082'], '경주월성(반월성)':['1237','1238'], 
    '경주첨성대':['1279','1280'], '경주금곡사':['5379','5380'],
    '경주기림사':['2021','2021'], '경주백률사':['1228','1229'],
    '경주분황사':['1157','1158'],'경주연지암':['5423','5426'], 
    '경주주사암':['3121','3122']
} 

In [ ]:
df

In [4]:
key_list = list(place_number.keys())
val_list = list(place_number.values())
change_type = []
for i in df['정류장번호']:
    change_type.append(str(i))

NameError: name 'df' is not defined

In [39]:
bus_df = pd.DataFrame()
for x in range(len(file_list)):
    df = pd.read_excel(file_list[x])
    del df['Unnamed: 0']

    key_list = list(place_number.keys())
    val_list = list(place_number.values())
    change_type = []

    for i in df['정류장번호']:
        change_type.append(str(i))

    result = file_list[x][10:].split('_')[0]

    for item in change_type:
        for i in range(len(val_list)):
            for j in range(2):
                if item == val_list[i][j]:
    #                 print(key_list[i], end=' ')
    #                 print(val_list[i][j])
                    data = {'bus': result, '정류장이름':key_list[i], 
                        '정류장번호':val_list[i][j]}
                    bus_df = bus_df.append(data, ignore_index=True)
bus_df

,bus,정류장이름,정류장번호
0,일반10,경주동궁과월지,1225
1,일반10,경주동부사적지대,1280
2,일반10,경주첨성대,1280
3,일반10,경주분황사,1158
4,일반10,경주불국사,1161
...,...,...,...
448,좌석700,경주동부사적지대,1280
449,좌석700,경주첨성대,1280
450,좌석700,경주월성(반월성),1238
451,좌석700,경주분황사,1158


In [370]:
bus_df['count']=1

In [381]:
bus_df_group = pd.DataFrame(bus_df.groupby(['정류장이름','정류장번호','bus']).count())
bus_df_group
bus_df_group.to_excel('bus_count.xlsx')

## 정류장별 시간 구하기

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings(action='ignore')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs

In [2]:
path='../../driver/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path))

In [3]:
# 네이버 지도 접속
driver.get('https://map.naver.com/')
time.sleep(3)

In [4]:
# df_time_list

In [ ]:
### <for문> 도착지에 세 번째(끝까지) 정류장 번호 입력 

# 폴더 내 버스 데이터 모두 불러오기

# df_time_list
file_list = glob.glob('./버스_data/새 폴더//새 폴더/*.xlsx')
# df_time = pd.read_excel(file_list[0])
# df_time_list = []
# for i in df_time['정류장번호']:
#     df_time_list.append(str(i).zfill(5))
    
# 데이터프레임 담는 리스트    
df_list = []

for i in range(len(file_list)):
    print(file_list[i])
    # 폴더 내 하나의 파일 불러와서 '정류장번호' string으로 변환 후 리스트 저장
    df_time = pd.read_excel(file_list[i])
    del df_time['Unnamed: 0']
    df_time_list = []
    for data in df_time['정류장번호']:
        df_time_list.append(str(data).zfill(5))
        
    # 배너 '길찾기 클릭'
    element = driver.find_element(By.CLASS_NAME,"link_navbar.directions")
    element.click()
    time.sleep(2)
    
    # '자동차' 버튼 클릭
    car_btn = driver.find_element(By.CLASS_NAME,"link_tab.car")
    car_btn.click()
    time.sleep(2)
        
    # 출발지:첫 번째 정류장 번호 입력 후 ENETER > '경상북도 경주'이면 클릭 
    dep_search = driver.find_element(By.ID, 'directionStart0')
    start_searh_name = df_time_list[0]
    if start_searh_name == '00미정차':
        continue
    dep_search.send_keys(start_searh_name)
    dep_search.send_keys(Keys.ENTER)
    time.sleep(3)
    geongbuk_button = driver.find_elements(By.CLASS_NAME, 'search_title')
    for count in range(len(geongbuk_button)):
        if geongbuk_button[count].text == df_time['버스노선'][0]:
            geongbuk_button[count].click()
    time.sleep(3)

    ##### 크롤링 #####
    for j in range(1, len(df_time_list)):
        
        # 도착지 > 두 번째 정류장 번호 입력 후 ENETER > '경상북도 경주'이면 클릭 
        arr_search = driver.find_element(By.ID, 'directionGoal1')
        arr_searh_name = df_time_list[j]
        if arr_searh_name == '00미정차':
            continue
        arr_search.send_keys(arr_searh_name)
        arr_search.send_keys(Keys.ENTER)
        time.sleep(3)
        geongbuk_button = driver.find_elements(By.CLASS_NAME, 'search_title')
        for z in range(len(geongbuk_button)):
            if geongbuk_button[z].text == df_time['버스노선'][0]:
                geongbuk_button[z].click()
        time.sleep(4)

        # 길찾기 버튼 클릭
        route_btn = driver.find_element(By.CLASS_NAME,"btn.btn_direction.active")
        route_btn.click()
        time.sleep(4)

        # '최적'에서 버스 그림에 있는 시간을 저장
        min_time = driver.find_elements(By.CLASS_NAME, "duration_time")
        min_time[0].text
        time.sleep(4)

        # 행 마다 도착시간 컬럼에 크롤링한 도착 시간 저장
        df_time['도착시간'].loc[j] = min_time[0].text

        # 출발지/도착지 전환 버튼 클릭
        change_btn = driver.find_element(By.CLASS_NAME,"btn_switch.ng-star-inserted")
        change_btn.click()
        time.sleep(4)

        # 도착지 이전 입력값 지우기
        arr_search.send_keys(Keys.CONTROL + "A")
        arr_search.send_keys(Keys.DELETE)
        time.sleep(3)
        
#     파일 이름 리스트로 저장
    name_list = []
    for item in range(len(file_list)):
        name_list.append(file_list[i].split('\\')[1].split('.')[0])
        
    df_list.append(df_time)
    df_time.to_excel(f"{name_list[i]}"+".xlsx")
    print(df_time)

./버스_data/새 폴더//새 폴더\일반231_3_05379.xlsx
            버스노선  정류장번호   버스번호   도착시간     배차간격
0        시외버스터미널   1224  일반231  06:25  평일 100분
1           보우주택   1148  일반231     1분  평일 100분
2          청년회의소   1322  일반231     1분  평일 100분
3           중앙시장   1307  일반231     1분  평일 100분
4     법원.동산병원 건너   1083  일반231     1분  평일 100분
5        신한은행사거리   1232  일반231     1분  평일 100분
6        성동시장 동편   1028  일반231     2분  평일 100분
7      세무서.큰마디병원   1211  일반231     1분  평일 100분
8      경주교.신라중학교   1013  일반231     1분  평일 100분
9         경주고용센타   1010  일반231     1분  평일 100분
10    황성공원.시립도서관   1381  일반231     1분  평일 100분
11        황성주공1차   1384  일반231     1분  평일 100분
12         계림중학교   1041  일반231     1분  평일 100분
13    황성주공2차.계림고   1385  일반231     1분  평일 100분
14       금장삼성아파트   5049  일반231     3분  평일 100분
15         금성아파트   5487  일반231     1분  평일 100분
16          금장2리   5047  일반231     1분  평일 100분
17    소현2리.현대자동차   5176  일반231     1분  평일 100분
18          서경주역   5309  일반231     2분  평일 100분
19          현곡농협   5

               버스노선  정류장번호   버스번호   도착시간        배차간격
0              고란종점   1487  일반234  06:15  평일 100분 1회
1              고란회관   1488  일반234     1분  평일 100분 1회
2             고란삼거리   1491  일반234     1분  평일 100분 1회
3              고란입구   5469  일반234     1분  평일 100분 1회
4                광명   1048  일반234     2분  평일 100분 1회
5              외외마을   1263  일반234     4분  평일 100분 1회
6            외외마을입구   1265  일반234     1분  평일 100분 1회
7         베스트원현대서비스   1138  일반234     1분  평일 100분 1회
8         문화중고등학교입구   1126  일반234     1분  평일 100분 1회
9     국민건강보험.정보고등학교   1060  일반234     1분  평일 100분 1회
10  서라벌대학기숙사앞.송화도서관   1187  일반234     1분  평일 100분 1회
11        서라벌대학.신라고   1186  일반234     1분  평일 100분 1회
12          경주초등학교앞   1036  일반234     1분  평일 100분 1회
13          경주여중건너편   1026  일반234     1분  평일 100분 1회
14     고속버스.시외버스터미널   1045  일반234     2분  평일 100분 1회
15      서라벌사거리.황리단길   1190  일반234     1분  평일 100분 1회
16             중앙시장   1309  일반234     1분  평일 100분 1회
17       법원.동산병원 건너   1083  일반234     1분  평일 1

       버스노선  정류장번호  버스번호   도착시간        배차간격
0      신경주역  03142  일반51  05:10  평일 20분 64회
1      옥수마을  03177  일반51     1분  평일 20분 64회
2       광명교    미정차  일반51  05:10  평일 20분 64회
3        광명  01048  일반51     4분  평일 20분 64회
4     남강주유소  01066  일반51     1분  평일 20분 64회
..      ...    ...   ...    ...         ...
66     와산마을  01259  일반51     1분  평일 20분 64회
67  농수산물집하장  01078  일반51     1분  평일 20분 64회
68       광명  01047  일반51     1분  평일 20분 64회
69     옥수마을  03176  일반51     3분  평일 20분 64회
70     신경주역  03143  일반51     1분  평일 20분 64회

[71 rows x 5 columns]
./버스_data/새 폴더//새 폴더\일반51_8_01036.xlsx
                버스노선  정류장번호  버스번호   도착시간       배차간격
0               신경주역  03142  일반51  05:10  평일 20분 1회
1               옥수마을  03177  일반51     1분  평일 20분 1회
2                광명교    미정차  일반51  05:10  평일 20분 1회
3                 광명  01048  일반51     4분  평일 20분 1회
4              남강주유소  01066  일반51     1분  평일 20분 1회
5               와산마을  01260  일반51     1분  평일 20분 1회
6              경주대학교  01016  일반51     1분  평일 2

                버스노선  정류장번호  버스번호   도착시간    배차간격
0               신경주역  03142  일반70  05:00  평일 15분
1               옥수마을  03177  일반70     1분  평일 15분
2                광명교    미정차  일반70  05:00  평일 15분
3                 광명  01048  일반70     4분  평일 15분
4              남강주유소  01066  일반70     1분  평일 15분
5               와산마을  01260  일반70     1분  평일 15분
6              경주대학교  01016  일반70     1분  평일 15분
7             외외마을입구  01265  일반70     1분  평일 15분
8          베스트원현대서비스  01138  일반70     1분  평일 15분
9          문화중고등학교입구  01126  일반70     1분  평일 15분
10     국민건강보험.정보고등학교  01060  일반70     1분  평일 15분
11   서라벌대학기숙사앞.송화도서관  01187  일반70     1분  평일 15분
12         서라벌대학.신라고  01186  일반70     1분  평일 15분
13           경주초등학교앞  01036  일반70     1분  평일 15분
14           경주여중건너편  01026  일반70     1분  평일 15분
15      고속버스.시외버스터미널  01045  일반70     2분  평일 15분
16       서라벌사거리.황리단길  01190  일반70     2분  평일 15분
17              중앙시장  01309  일반70     1분  평일 15분
18        법원.동산병원 건너  01083  일반70     1분  평일 15분
19           신한은행사거리

                버스노선  정류장번호  버스번호   도착시간    배차간격
0            문화중고등학교   1124  일반70  05:00  평일 15분
1              새천년미소   1304  일반70     1분  평일 15분
2               충효지구   1329  일반70     1분  평일 15분
3          문화중고등학교입구   1126  일반70     2분  평일 15분
4      국민건강보험.정보고등학교   1060  일반70     1분  평일 15분
5    서라벌대학기숙사앞.송화도서관   1187  일반70     1분  평일 15분
6          서라벌대학.신라고   1186  일반70     1분  평일 15분
7            경주초등학교앞   1036  일반70     1분  평일 15분
8            경주여중건너편   1026  일반70     1분  평일 15분
9       고속버스.시외버스터미널   1045  일반70     2분  평일 15분
10       서라벌사거리.황리단길   1190  일반70     2분  평일 15분
11              중앙시장   1309  일반70     1분  평일 15분
12        법원.동산병원 건너   1083  일반70     1분  평일 15분
13           신한은행사거리   1232  일반70     2분  평일 15분
14           성동시장 동편   1028  일반70     1분  평일 15분
15         세무서.큰마디병원   1211  일반70     1분  평일 15분
16         경주교.신라중학교   1014  일반70     1분  평일 15분
17          동보아파트맞은편   1111  일반70     1분  평일 15분
18             선주아파트   1204  일반70     1분  평일 15분
19               알천교

                버스노선  정류장번호  버스번호   도착시간    배차간격
0               신경주역  03142  일반70  05:00  평일 15분
1               옥수마을  03177  일반70     1분  평일 15분
2                광명교    미정차  일반70  05:00  평일 15분
3                 광명  01048  일반70     4분  평일 15분
4              남강주유소  01066  일반70     1분  평일 15분
5               와산마을  01260  일반70     1분  평일 15분
6              경주대학교  01016  일반70     1분  평일 15분
7             외외마을입구  01265  일반70     1분  평일 15분
8          베스트원현대서비스  01138  일반70     1분  평일 15분
9          문화중고등학교입구  01126  일반70     1분  평일 15분
10     국민건강보험.정보고등학교  01060  일반70     1분  평일 15분
11   서라벌대학기숙사앞.송화도서관  01187  일반70     1분  평일 15분
12         서라벌대학.신라고  01186  일반70     1분  평일 15분
13           경주초등학교앞  01036  일반70     1분  평일 15분
14           경주여중건너편  01026  일반70     1분  평일 15분
15      고속버스.시외버스터미널  01045  일반70     1분  평일 15분
16       서라벌사거리.황리단길  01190  일반70     1분  평일 15분
17              중앙시장  01309  일반70     1분  평일 15분
18        법원.동산병원 건너  01083  일반70     1분  평일 15분
19           신한은행사거리

In [124]:
ai = pd.read_excel('./버스_data/새 폴더/일반50_4_01025.xlsx')
ai[ai['정류장번호']=='미정차']

,Unnamed: 0,버스노선,정류장번호,버스번호,도착시간,배차간격
2,2,광명교,미정차,일반50,05:00,평일 20분 65회


In [125]:
start_searh_name = df_time_list[0]
if start_searh_name == '미정차':
    continue

SyntaxError: 'continue' not properly in loop (1869327131.py, line 3)

In [127]:
df_time_list

['03142',
 '03177',
 '00미정차',
 '01048',
 '01066',
 '01260',
 '01016',
 '01265',
 '01138',
 '01126',
 '01060',
 '01187',
 '01186',
 '01036',
 '01026',
 '01045',
 '01190',
 '01309',
 '01083',
 '01232',
 '01028',
 '01211',
 '01013',
 '01010',
 '01381',
 '01089',
 '01270',
 '01062',
 '01276',
 '01198',
 '01275',
 '01267',
 '01446',
 '01444',
 '01489',
 '01001',
 '01448',
 '01268',
 '01061',
 '01134',
 '01325',
 '01118',
 '01385',
 '05049',
 '05044',
 '01104',
 '01103',
 '01209',
 '01021',
 '01206',
 '01151',
 '01008',
 '01308',
 '01189',
 '01044',
 '01025',
 '01037',
 '01185',
 '01188',
 '01059',
 '01127',
 '01239',
 '01264',
 '01017',
 '01259',
 '01078',
 '01047',
 '03176',
 '03143']